In [1]:
from newsapi import NewsApiClient
from nytimesarticle import articleAPI
import pandas_datareader.data as web
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dateutil
import time
import quandl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas.plotting import autocorrelation_plot
from pandas.tools.plotting import lag_plot
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

//anaconda/envs/py36/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
//anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [43]:
# Params
# extra API keys, comment out as necessary
#api = articleAPI("b23351c6f9314694bfe4f4929a2b72c5") 
#api = articleAPI("787bd4db8e704bbf9cebe8b7941827e0") 
#api = articleAPI("f8b402f42ed14b249fd5accc95a050dd") 
#api = articleAPI("c91a676aeaef40fd844409c8b0bef485")
#api = articleAPI("c43133d654134109868299ff505e7c55")
#api = articleAPI("eb427ebc2336423ead4d350cfa4e900b")
api = articleAPI("b538de93f1a9459da22b150d7b53cb6f")
#api = articleAPI("88f587ed149d4478b4490168d61ed9dc")

quandl.ApiConfig.api_key = "2S7d7eeL5VZrLup9pKg5"
end_date = (datetime.datetime.now() - datetime.timedelta(days=3)).isoformat()
start_date = (datetime.datetime.now() - datetime.timedelta(days=365)).isoformat()
left_sources = 'The New York Times'
right_sources = 'Fox News'
center_sources = 'Reuters AP The Wall Street Journal'
all_sources = left_sources + ' ' + right_sources + ' ' + center_sources
replace_list = ['Corp', 'Inc.', 'Inc', '.com', 'plc', ',', 'Co.']
# domain

In [29]:
df = pd.read_csv('constituents_csv.csv')
companies = df['Name']
company_symb = {}

In [30]:
df.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [41]:
df[df['Name'] == 'Amazon.com Inc.']

,Symbol,Name,Sector
30,AMZN,Amazon.com Inc.,Consumer Discretionary


In [ ]:
# company = companies.loc[i]
company_hits = []
start_date = int(start_date.replace("-","").split('T')[0])
ending_date = (dateutil.parser.parse(end_date)-datetime.timedelta(days=1)).isoformat()
end_date = int(ending_date.replace("-","").split('T')[0])

for company in companies:
    try:
        # get company ticker
        company_symb[company] = df[df['Name'] == company]['Symbol']
        ticker = company_symb[company].values[0]

        # get rid of suffixes from company name
        for word in replace_list:
            company = company.replace(word, '')

        newsdata = api.search(q=company, begin_date = start_date,
                               end_date = end_date,
                              fq='headline:('+company+ ') OR body:('+company+') AND source:(' + all_sources + ')',
                              page = 0,
                              facet_filter = True)

        number_of_hits = newsdata['response']['meta']['hits']
        company_hits.append(number_of_hits)
        time.sleep(1)
    except:
        company_hits.append(0)
        print(company, 'failed')
        time.sleep(1)
    

3m company
headline:(3m company) or body:(3m company) and source:(the new york times fox news reuters ap the wall street journal)
a.o. smith 
headline:(a.o. smith ) or body:(a.o. smith ) and source:(the new york times fox news reuters ap the wall street journal)
abbott laboratories
headline:(abbott laboratories) or body:(abbott laboratories) and source:(the new york times fox news reuters ap the wall street journal)
abbvie 
headline:(abbvie ) or body:(abbvie ) and source:(the new york times fox news reuters ap the wall street journal)
accenture 
headline:(accenture ) or body:(accenture ) and source:(the new york times fox news reuters ap the wall street journal)
activision blizzard
headline:(activision blizzard) or body:(activision blizzard) and source:(the new york times fox news reuters ap the wall street journal)
acuity brands 
headline:(acuity brands ) or body:(acuity brands ) and source:(the new york times fox news reuters ap the wall street journal)
adobe systems 
headline:(adobe

In [42]:
print((company_hits))
print(sum(i > 4000 for i in company_hits))
print(company_hits[30])

[31, 22, 51, 31, 58, 28, 3, 24, 26, 22, 7, 193, 21, 4, 4, 0, 10, 83, 18, 100, 8, 100, 1, 32, 61, 1, 42, 85, 10, 36, 4125, 1, 368, 560, 306, 4641, 1364, 3877, 6, 9, 0, 47, 0, 17, 30, 12, 1, 1558, 36, 0, 0, 4111, 167, 0, 137, 35, 0, 4, 0, 12, 17, 4, 5, 1, 77, 7248, 1961, 16, 0, 0, 274, 3470, 28, 249, 0, 419, 19, 5, 119, 164, 2, 64, 142, 20, 29, 204, 0, 4, 45, 2600, 63, 11, 573, 145, 13, 22, 2142, 27, 0, 3, 43, 4, 3, 0, 133, 240, 37, 6, 0, 80, 0, 127, 0, 87, 598, 497, 0, 54, 3, 219, 7, 107, 409, 40, 5, 5, 39, 42, 23, 36, 33, 26, 21, 5, 34, 50, 112, 11, 7, 17, 12, 0, 376, 0, 50, 12, 64, 266, 35, 113, 950, 132, 27, 252, 12, 37, 6, 163, 7, 2, 539, 5, 0, 355, 2, 0, 0, 632, 13, 17, 12, 8, 0, 158, 3, 81, 2, 5, 20, 14, 23, 103, 0, 61, 80, 495, 1, 10779, 1, 289, 0, 1164, 7, 19, 3, 2, 0, 16, 0, 248, 452, 1, 0, 103, 96, 3547, 1, 100, 246, 1077, 90, 242, 878, 190, 29, 615, 776, 0, 0, 54, 6, 634, 1, 0, 63, 2, 2, 0, 4, 0, 78, 21, 6, 371, 0, 10, 0, 91, 126, 2, 7, 1, 5, 24, 9, 0, 1, 487, 78, 273, 1510, 

In [ ]:
def query_news_articles(company, start_date, end_date, trading_dates, sources):
    """ Queries news article for a certain time frame and split it by dates
        Note that
    Params:
        company (String): Name of company
        start_date (String): Start date in format of "2001-12-31"
        end_date (String): End date in format of "2001-12-31"
         trading_dates (Array of Strings): Array of dates when the market was open
                 dates in format of "2001-12-31"
        sources (Array of Strings): Array of different news sources
    Returns:
        company_dic (dictionary): keys are date, values are array of headlines
    """
    company_dict = {k: [] for k in trading_dates.date}
    start_date = int(start_date.replace("-","").split('T')[0])
    ending_date = (dateutil.parser.parse(end_date)-datetime.timedelta(days=1)).isoformat()
    end_date = int(ending_date.replace("-","").split('T')[0])
    newsdata = api.search(q=company, begin_date = start_date,
                           end_date = end_date,
                          fq='headline:('+company+ ') OR body:('+company+') AND source:(' + sources + ')',
                          page = 0,
                          facet_filter = True)
                               

    #print(newsdata) # newsdata is full HTTP response
    number_of_hits = newsdata['response']['meta']['hits']
    number_of_pages = (number_of_hits // 10) + 1
    
    time.sleep(1)
    # page through results and add headlines to companY_dict
    for i in range(0, min(number_of_pages,200)):
        print('page', i)
        newsdata = api.search(q=company, begin_date = start_date,
                           end_date = end_date,
                          fq='headline:('+company+ ') OR body:('+company+') AND source:(' + sources + ')',
                          page = i,
                          facet_filter = True)
        articles = newsdata['response']['docs']
        for article in articles:
            relevance = article['score']
            if relevance >= 0.005: 
                headline = article['headline']['main']
                blurb = article['snippet']
                # print(article['pub_date'], '\t', article['headline']['main'])
            
                # description = article['description']
                # format of date is 2018-04-13T00:46:59Z (UTC format)
                publish_date = article['pub_date'] 
                print(publish_date)
                # adjust date for trading day
                publish_date, publish_time = publish_date.split('T')
                date_arr = publish_date.split('-')
                publish_datetime = datetime.date(int(date_arr[0]), int(date_arr[1]), int(date_arr[2]))
                time_arr = publish_time[:-1].split(':')
                # stock market closes at 4:00 PM EST; if article published after 
                # 16:00:00+4:00:00 = 20:00:00 UTC headline affects next trading day;
                # otherwise affects current trading day
                trading_datetime = publish_datetime
                if int(time_arr[0]) >= 20:
                    trading_datetime += datetime.timedelta(days=1)
                
                # if given trading_date invalid (ie if article published on Friday 
                # after market close, Saturday, or Sunday before 4 pm est) push trading_date
                # to the following Monday (ie first valid trading_date)
                while trading_datetime not in trading_dates:
                    trading_datetime += datetime.timedelta(1)
                company_dict[trading_datetime].append(headline)
                # company_dict[trading_datetime].append(blurb) include 'snippet' in sentiment analysis
        time.sleep(1)
        
    return company_dict